In [2]:
import tensorflow as tf

/home/jenno/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def model_fn(features, labels, mode, params):
    x = tf.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu')(features['image'])
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (5, 5), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x_out = tf.keras.layers.Dense(10, activation= 'softmax')(x)
    prediction_cls = tf.argmax(x_out, -1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        spec = tf.estimator.EstimatorSpec(mode=mode, predictions= prediction_cls)
    else:
        one_hot_labels = tf.one_hot(labels, 10)
        cross_entropy = tf.keras.backend.categorical_crossentropy(one_hot_labels, x_out)
        loss = tf.reduce_mean(cross_entropy)
        optimizer = tf.train.AdadeltaOptimizer(0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        metrics = {"accuracy": tf.metrics.accuracy(labels, prediction_cls)}
        spec = tf.estimator.EstimatorSpec(mode= mode, loss=loss, train_op=train_op, eval_metric_ops=metrics)
    return spec

In [4]:
def read_and_decode(record):
    keys_to_features = {
        "image_raw": tf.FixedLenFeature([], tf.string),
        "label":     tf.FixedLenFeature([], tf.int64)
    }
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed['image_raw'], tf.uint8)
    label = parsed['label']
    image = tf.cast(image, tf.float32)
    image = image / 255
    image = tf.reshape(image, [28, 28, 1])
    return {"image": image}, label

def _input_fn(mode, data_dir):
    dataset = tf.data.TFRecordDataset(data_dir, num_parallel_reads= 4)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(60000))
    dataset = dataset.apply(tf.contrib.data.map_and_batch(read_and_decode, 128))
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
    return dataset

def train_input_fn():
    return _input_fn(tf.estimator.ModeKeys.TRAIN, data_dir = '/home/jenno/Desktop/data/mnist/train.tfrecords')

def eval_input_fn():
    return _input_fn(tf.estimator.ModeKeys.EVAL, data_dir = '/home/jenno/Desktop/data/mnist/test.tfrecords')
    
    

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model = tf.estimator.Estimator(model_fn= model_fn)
    model.train(input_fn = train_input_fn, steps = 40000)
    results = model.evaluate(input_fn=eval_input_fn, steps = 100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpevr58akl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3510f40fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpevr58akl/model.ckpt.
INFO:ten

INFO:tensorflow:loss = 1.4626722, step = 7100 (0.827 sec)
INFO:tensorflow:global_step/sec: 120.993
INFO:tensorflow:loss = 1.505048, step = 7200 (0.827 sec)
INFO:tensorflow:global_step/sec: 120.733
INFO:tensorflow:loss = 1.5037591, step = 7300 (0.828 sec)
INFO:tensorflow:global_step/sec: 119.57
INFO:tensorflow:loss = 1.4262023, step = 7400 (0.836 sec)
INFO:tensorflow:global_step/sec: 121.135
INFO:tensorflow:loss = 1.3959439, step = 7500 (0.826 sec)
INFO:tensorflow:global_step/sec: 120.395
INFO:tensorflow:loss = 1.4571198, step = 7600 (0.831 sec)
INFO:tensorflow:global_step/sec: 120.891
INFO:tensorflow:loss = 1.348002, step = 7700 (0.827 sec)
INFO:tensorflow:global_step/sec: 120.209
INFO:tensorflow:loss = 1.3673475, step = 7800 (0.832 sec)
INFO:tensorflow:global_step/sec: 120.82
INFO:tensorflow:loss = 1.3263959, step = 7900 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.541
INFO:tensorflow:loss = 1.2492766, step = 8000 (0.822 sec)
INFO:tensorflow:global_step/sec: 120.841
INFO:tensorflo

INFO:tensorflow:global_step/sec: 121.232
INFO:tensorflow:loss = 0.58488965, step = 15400 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.966
INFO:tensorflow:loss = 0.63307625, step = 15500 (0.820 sec)
INFO:tensorflow:global_step/sec: 119.912
INFO:tensorflow:loss = 0.5773382, step = 15600 (0.834 sec)
INFO:tensorflow:global_step/sec: 119.748
INFO:tensorflow:loss = 0.5423809, step = 15700 (0.835 sec)
INFO:tensorflow:global_step/sec: 120.902
INFO:tensorflow:loss = 0.5463729, step = 15800 (0.827 sec)
INFO:tensorflow:global_step/sec: 120.767
INFO:tensorflow:loss = 0.4808515, step = 15900 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.375
INFO:tensorflow:loss = 0.70205164, step = 16000 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.021
INFO:tensorflow:loss = 0.57244796, step = 16100 (0.826 sec)
INFO:tensorflow:global_step/sec: 121.078
INFO:tensorflow:loss = 0.584687, step = 16200 (0.826 sec)
INFO:tensorflow:global_step/sec: 119.891
INFO:tensorflow:loss = 0.63176334, step = 16300 (0.834 s

INFO:tensorflow:global_step/sec: 121.037
INFO:tensorflow:loss = 0.38267732, step = 23600 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.075
INFO:tensorflow:loss = 0.4194048, step = 23700 (0.826 sec)
INFO:tensorflow:global_step/sec: 120.823
INFO:tensorflow:loss = 0.35819912, step = 23800 (0.829 sec)
INFO:tensorflow:global_step/sec: 120.075
INFO:tensorflow:loss = 0.41948992, step = 23900 (0.831 sec)
INFO:tensorflow:global_step/sec: 121.381
INFO:tensorflow:loss = 0.43075648, step = 24000 (0.824 sec)
INFO:tensorflow:global_step/sec: 120.866
INFO:tensorflow:loss = 0.39394644, step = 24100 (0.828 sec)
INFO:tensorflow:global_step/sec: 120.247
INFO:tensorflow:loss = 0.42900077, step = 24200 (0.832 sec)
INFO:tensorflow:global_step/sec: 120.942
INFO:tensorflow:loss = 0.40545076, step = 24300 (0.827 sec)
INFO:tensorflow:global_step/sec: 120.315
INFO:tensorflow:loss = 0.44802922, step = 24400 (0.831 sec)
INFO:tensorflow:global_step/sec: 120.699
INFO:tensorflow:loss = 0.46085533, step = 24500 (0.

INFO:tensorflow:global_step/sec: 120.422
INFO:tensorflow:loss = 0.26460165, step = 31800 (0.830 sec)
INFO:tensorflow:global_step/sec: 120.763
INFO:tensorflow:loss = 0.30876037, step = 31900 (0.828 sec)
INFO:tensorflow:global_step/sec: 120.837
INFO:tensorflow:loss = 0.28521174, step = 32000 (0.827 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 0.29431686, step = 32100 (0.825 sec)
INFO:tensorflow:global_step/sec: 119.664
INFO:tensorflow:loss = 0.3042516, step = 32200 (0.836 sec)
INFO:tensorflow:global_step/sec: 120.718
INFO:tensorflow:loss = 0.42249283, step = 32300 (0.828 sec)
INFO:tensorflow:global_step/sec: 120.777
INFO:tensorflow:loss = 0.44891614, step = 32400 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.545
INFO:tensorflow:loss = 0.29471713, step = 32500 (0.823 sec)
INFO:tensorflow:global_step/sec: 120.643
INFO:tensorflow:loss = 0.31514084, step = 32600 (0.829 sec)
INFO:tensorflow:global_step/sec: 120.687
INFO:tensorflow:loss = 0.39639157, step = 32700 (0.

INFO:tensorflow:loss = 0.33777174, step = 39900 (0.812 sec)
INFO:tensorflow:Saving checkpoints for 40000 into /tmp/tmpevr58akl/model.ckpt.
INFO:tensorflow:Loss for final step: 0.30431187.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-20-07:34:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpevr58akl/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Finished evaluation at 2018-07-20-07:34:30
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9214, global_step = 40000, loss = 0.27819628
INFO:tensorflow:Saving 'checkpoint_path' summary for global step